# ML Enhancement: Promotion Effectiveness Analysis

Analyzes price elasticity and promotional lift to optimize marketing spend.

## Business Objective
- Identify which promotions drive incremental sales
- Understand price sensitivity by product category
- Measure promotional ROI and cannibalization effects
- Optimize discount levels and promotional calendar

## Method
- **Price Elasticity**: Log-log regression of quantity vs price
- **Incremental Lift**: Compare promoted vs. baseline sales periods
- **Cannibalization**: Measure sales decline in adjacent periods

## Data Flow
```
fact_receipt_lines + fact_promotions + dim_products
  --> gold_price_elasticity (product-level)
  --> gold_promotion_lift (promotion-level)
```

## Usage
Run this notebook after historical data load to establish baseline metrics.
Re-run periodically (weekly/monthly) to track promotional effectiveness trends.

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
import os
import numpy as np
from typing import Tuple

In [ ]:
# =============================================================================
# PARAMETERS
# =============================================================================

def get_env(var_name: str, default: str | None = None) -> str | None:
    return os.environ.get(var_name, default)

SILVER_DB = get_env("SILVER_DB", default="ag")
GOLD_DB = get_env("GOLD_DB", default="au")

# Analysis parameters
MIN_OBSERVATIONS = 30  # Minimum data points for elasticity calculation
BASELINE_WINDOW_DAYS = 30  # Days before/after promo for baseline
TOP_N_PRODUCTS = 100  # Top products by revenue to analyze

print(f"Configuration: SILVER_DB={SILVER_DB}, GOLD_DB={GOLD_DB}")
print(f"Analysis Parameters:")
print(f"  MIN_OBSERVATIONS: {MIN_OBSERVATIONS}")
print(f"  BASELINE_WINDOW_DAYS: {BASELINE_WINDOW_DAYS}")
print(f"  TOP_N_PRODUCTS: {TOP_N_PRODUCTS}")

In [ ]:
# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def ensure_database(name: str) -> None:
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {name}")

def read_silver(table_name: str):
    return spark.table(f"{SILVER_DB}.{table_name}")

def save_gold(df, table_name: str) -> None:
    full_name = f"{GOLD_DB}.{table_name}"
    df.write.format("delta").mode("overwrite").saveAsTable(full_name)
    print(f"  {full_name}: {df.count()} rows")

def silver_exists(table_name: str) -> bool:
    try:
        spark.table(f"{SILVER_DB}.{table_name}")
        return True
    except AnalysisException:
        return False

ensure_database(GOLD_DB)

---
## Part 1: Data Preparation

Join receipt lines with product dimensions and promotion data.

In [ ]:
print("="*60)
print("DATA PREPARATION")
print("="*60)

# Check required tables exist
required_tables = ["fact_receipt_lines", "dim_products", "fact_promotions"]
missing_tables = [t for t in required_tables if not silver_exists(t)]

if missing_tables:
    print(f"ERROR: Missing required tables: {missing_tables}")
    print("Cannot proceed with analysis.")
    raise Exception(f"Missing required tables: {missing_tables}")

print("All required tables present.")

In [ ]:
# Load dimension and fact tables
df_receipt_lines = read_silver("fact_receipt_lines")
df_products = read_silver("dim_products")
df_promotions = read_silver("fact_promotions")

print(f"Receipt lines: {df_receipt_lines.count():,} rows")
print(f"Products: {df_products.count():,} rows")
print(f"Promotions: {df_promotions.count():,} rows")

In [ ]:
# Enrich receipt lines with product attributes
df_sales = (
    df_receipt_lines
    .join(df_products, on="product_id", how="inner")
    .select(
        "event_ts",
        "receipt_id_ext",
        "product_id",
        F.col("name").alias("product_name"),
        F.col("category").alias("product_category"),
        F.col("subcategory").alias("product_subcategory"),
        "quantity",
        "unit_price",
        "ext_price",
        "promo_code",
        F.col("base_price").alias("regular_price"),
        F.col("base_cost").alias("product_cost")
    )
    .withColumn("date", F.to_date("event_ts"))
    .withColumn(
        "is_promoted",
        F.when(F.col("promo_code").isNotNull(), 1).otherwise(0)
    )
    .withColumn(
        "discount_pct",
        F.when(
            (F.col("regular_price") > 0) & (F.col("unit_price") < F.col("regular_price")),
            ((F.col("regular_price") - F.col("unit_price")) / F.col("regular_price")) * 100
        ).otherwise(0.0)
    )
)

print(f"\nEnriched sales data: {df_sales.count():,} rows")
df_sales.printSchema()

---
## Part 2: Price Elasticity Estimation

Calculate price elasticity using log-log regression:
```
log(quantity) = β₀ + β₁ * log(price) + ε
```
Where β₁ is the price elasticity coefficient.

In [ ]:
print("="*60)
print("PRICE ELASTICITY ANALYSIS")
print("="*60)

# Identify top products by revenue for analysis
df_top_products = (
    df_sales
    .groupBy("product_id", "product_name", "product_category")
    .agg(
        F.sum("ext_price").alias("total_revenue"),
        F.sum("quantity").alias("total_units"),
        F.countDistinct("date").alias("days_sold")
    )
    .filter(F.col("days_sold") >= MIN_OBSERVATIONS)
    .orderBy(F.desc("total_revenue"))
    .limit(TOP_N_PRODUCTS)
)

print(f"\nAnalyzing top {TOP_N_PRODUCTS} products with >= {MIN_OBSERVATIONS} days of sales data")
df_top_products.show(10, truncate=False)

In [ ]:
# Aggregate sales by product and date for elasticity calculation
df_daily_sales = (
    df_sales
    .join(
        df_top_products.select("product_id"),
        on="product_id",
        how="inner"
    )
    .groupBy("product_id", "product_name", "product_category", "date")
    .agg(
        F.sum("quantity").alias("daily_quantity"),
        F.avg("unit_price").alias("avg_price"),
        F.max("regular_price").alias("regular_price"),
        F.max("is_promoted").alias("is_promoted")
    )
    .filter(
        (F.col("daily_quantity") > 0) &
        (F.col("avg_price") > 0)
    )
    .withColumn("log_quantity", F.log(F.col("daily_quantity")))
    .withColumn("log_price", F.log(F.col("avg_price")))
)

print(f"\nDaily sales aggregation: {df_daily_sales.count():,} product-day observations")

In [ ]:
# Calculate elasticity statistics per product using aggregation
# Elasticity = covariance(log_price, log_quantity) / variance(log_price)

df_elasticity_stats = (
    df_daily_sales
    .groupBy("product_id", "product_name", "product_category")
    .agg(
        F.count("*").alias("n_observations"),
        F.avg("log_price").alias("mean_log_price"),
        F.avg("log_quantity").alias("mean_log_quantity"),
        F.stddev("log_price").alias("stddev_log_price"),
        F.stddev("log_quantity").alias("stddev_log_quantity"),
        F.avg("avg_price").alias("avg_price"),
        F.avg("regular_price").alias("regular_price"),
        F.avg("daily_quantity").alias("avg_daily_quantity")
    )
)

print(f"\nElasticity statistics calculated for {df_elasticity_stats.count():,} products")

In [ ]:
# Join back to calculate covariance and elasticity
df_elasticity_calc = (
    df_daily_sales
    .join(df_elasticity_stats, on=["product_id", "product_name", "product_category"], how="inner")
    .withColumn(
        "price_deviation",
        F.col("log_price") - F.col("mean_log_price")
    )
    .withColumn(
        "quantity_deviation",
        F.col("log_quantity") - F.col("mean_log_quantity")
    )
    .withColumn(
        "cross_product",
        F.col("price_deviation") * F.col("quantity_deviation")
    )
    .withColumn(
        "price_sq_deviation",
        F.col("price_deviation") * F.col("price_deviation")
    )
)

df_price_elasticity = (
    df_elasticity_calc
    .groupBy("product_id", "product_name", "product_category")
    .agg(
        F.first("n_observations").alias("n_observations"),
        F.first("avg_price").alias("avg_price"),
        F.first("regular_price").alias("regular_price"),
        F.first("avg_daily_quantity").alias("avg_daily_quantity"),
        (F.sum("cross_product") / F.sum("price_sq_deviation")).alias("elasticity_coefficient"),
        F.first("stddev_log_price").alias("stddev_log_price"),
        F.first("stddev_log_quantity").alias("stddev_log_quantity")
    )
    .withColumn(
        "elasticity_category",
        F.when(F.abs(F.col("elasticity_coefficient")) > 1.5, "Highly Elastic")
         .when(F.abs(F.col("elasticity_coefficient")) > 1.0, "Elastic")
         .when(F.abs(F.col("elasticity_coefficient")) > 0.5, "Unit Elastic")
         .otherwise("Inelastic")
    )
    .withColumn(
        "standard_error",
        F.col("stddev_log_quantity") / F.sqrt(F.col("n_observations"))
    )
    .withColumn(
        "confidence_interval_lower",
        F.col("elasticity_coefficient") - (1.96 * F.col("standard_error"))
    )
    .withColumn(
        "confidence_interval_upper",
        F.col("elasticity_coefficient") + (1.96 * F.col("standard_error"))
    )
    .withColumn("computed_at", F.current_timestamp())
    .select(
        "product_id",
        "product_name",
        "product_category",
        "elasticity_coefficient",
        "elasticity_category",
        "confidence_interval_lower",
        "confidence_interval_upper",
        "n_observations",
        "avg_price",
        "regular_price",
        "avg_daily_quantity",
        "computed_at"
    )
)

print(f"\nPrice elasticity calculated for {df_price_elasticity.count():,} products")
print("\nSample results:")
df_price_elasticity.orderBy(F.abs(F.col("elasticity_coefficient")), ascending=False).show(10, truncate=False)

In [ ]:
# Save price elasticity to Gold layer
print("\nSaving gold_price_elasticity...")
save_gold(df_price_elasticity, "gold_price_elasticity")

---
## Part 3: Promotion Lift Analysis

Measure incremental sales lift from promotions by comparing:
- Baseline sales (non-promoted periods)
- Promoted sales (during promotion)
- Post-promotion sales (cannibalization effect)

In [ ]:
print("="*60)
print("PROMOTION LIFT ANALYSIS")
print("="*60)

# Aggregate promotion events by promo_code and date range
df_promo_periods = (
    df_promotions
    .withColumn("promo_date", F.to_date("event_ts"))
    .groupBy("promo_code", "discount_type")
    .agg(
        F.min("promo_date").alias("promo_start_date"),
        F.max("promo_date").alias("promo_end_date"),
        F.countDistinct("receipt_id").alias("promoted_receipts"),
        F.avg("discount_amount").alias("avg_discount")
    )
    .withColumn(
        "baseline_start_date",
        F.date_sub(F.col("promo_start_date"), BASELINE_WINDOW_DAYS)
    )
    .withColumn(
        "baseline_end_date",
        F.date_sub(F.col("promo_start_date"), 1)
    )
    .withColumn(
        "post_promo_start_date",
        F.date_add(F.col("promo_end_date"), 1)
    )
    .withColumn(
        "post_promo_end_date",
        F.date_add(F.col("promo_end_date"), BASELINE_WINDOW_DAYS)
    )
)

print(f"\nPromotion periods identified: {df_promo_periods.count():,}")
df_promo_periods.show(10, truncate=False)

In [ ]:
# Calculate baseline sales (pre-promotion)
df_sales_with_promo = df_sales.alias("sales").join(
    df_promo_periods.alias("promo"),
    F.col("sales.promo_code") == F.col("promo.promo_code"),
    how="inner"
)

df_baseline_sales = (
    df_sales_with_promo
    .filter(
        (F.col("sales.date") >= F.col("promo.baseline_start_date")) &
        (F.col("sales.date") <= F.col("promo.baseline_end_date"))
    )
    .groupBy("promo.promo_code", "sales.product_id")
    .agg(
        F.sum("sales.quantity").alias("baseline_quantity"),
        F.sum("sales.ext_price").alias("baseline_revenue"),
        F.countDistinct("sales.date").alias("baseline_days")
    )
    .withColumn(
        "baseline_daily_quantity",
        F.col("baseline_quantity") / F.col("baseline_days")
    )
    .withColumnRenamed("promo_code", "baseline_promo_code")
    .withColumnRenamed("product_id", "baseline_product_id")
)

print(f"\nBaseline sales calculated: {df_baseline_sales.count():,} promo-product combinations")

In [ ]:
# Calculate promoted sales
df_promoted_sales = (
    df_sales_with_promo
    .filter(
        (F.col("sales.date") >= F.col("promo.promo_start_date")) &
        (F.col("sales.date") <= F.col("promo.promo_end_date"))
    )
    .groupBy("promo.promo_code", "sales.product_id")
    .agg(
        F.sum("sales.quantity").alias("promo_quantity"),
        F.sum("sales.ext_price").alias("promo_revenue"),
        F.countDistinct("sales.date").alias("promo_days"),
        F.avg("sales.discount_pct").alias("avg_discount_pct")
    )
    .withColumn(
        "promo_daily_quantity",
        F.col("promo_quantity") / F.col("promo_days")
    )
    .withColumnRenamed("promo_code", "promo_promo_code")
    .withColumnRenamed("product_id", "promo_product_id")
)

print(f"\nPromoted sales calculated: {df_promoted_sales.count():,} promo-product combinations")

In [ ]:
# Calculate post-promotion sales (cannibalization)
df_post_promo_sales = (
    df_sales_with_promo
    .filter(
        (F.col("sales.date") >= F.col("promo.post_promo_start_date")) &
        (F.col("sales.date") <= F.col("promo.post_promo_end_date"))
    )
    .groupBy("promo.promo_code", "sales.product_id")
    .agg(
        F.sum("sales.quantity").alias("post_quantity"),
        F.sum("sales.ext_price").alias("post_revenue"),
        F.countDistinct("sales.date").alias("post_days")
    )
    .withColumn(
        "post_daily_quantity",
        F.col("post_quantity") / F.col("post_days")
    )
    .withColumnRenamed("promo_code", "post_promo_code")
    .withColumnRenamed("product_id", "post_product_id")
)

print(f"\nPost-promotion sales calculated: {df_post_promo_sales.count():,} promo-product combinations")

In [ ]:
# Combine and calculate lift metrics
df_promotion_lift = (
    df_promoted_sales
    .join(
        df_baseline_sales,
        (F.col("promo_promo_code") == F.col("baseline_promo_code")) &
        (F.col("promo_product_id") == F.col("baseline_product_id")),
        how="left"
    )
    .join(
        df_post_promo_sales,
        (F.col("promo_promo_code") == F.col("post_promo_code")) &
        (F.col("promo_product_id") == F.col("post_product_id")),
        how="left"
    )
    .join(
        df_products.select(
            F.col("product_id"),
            F.col("name").alias("product_name"),
            F.col("category").alias("product_category")
        ),
        F.col("promo_product_id") == F.col("product_id"),
        how="inner"
    )
    .join(
        df_promo_periods.select(
            F.col("promo_code"),
            F.col("discount_type"),
            F.col("avg_discount"),
            F.col("promo_start_date"),
            F.col("promo_end_date")
        ),
        F.col("promo_promo_code") == F.col("promo_code"),
        how="inner"
    )
    .select(
        F.col("promo_code"),
        F.col("promo_product_id").alias("product_id"),
        "product_name",
        "product_category",
        "discount_type",
        "avg_discount",
        "avg_discount_pct",
        "promo_start_date",
        "promo_end_date",
        F.coalesce(F.col("baseline_daily_quantity"), F.lit(0.0)).alias("baseline_daily_quantity"),
        F.col("promo_daily_quantity"),
        F.coalesce(F.col("post_daily_quantity"), F.lit(0.0)).alias("post_daily_quantity"),
        F.col("promo_quantity").alias("total_promoted_quantity"),
        F.col("promo_revenue").alias("total_promoted_revenue")
    )
    .withColumn(
        "incremental_lift_pct",
        F.when(
            F.col("baseline_daily_quantity") > 0,
            ((F.col("promo_daily_quantity") - F.col("baseline_daily_quantity")) / 
             F.col("baseline_daily_quantity")) * 100
        ).otherwise(F.lit(None))
    )
    .withColumn(
        "cannibalization_pct",
        F.when(
            F.col("baseline_daily_quantity") > 0,
            ((F.col("baseline_daily_quantity") - F.col("post_daily_quantity")) / 
             F.col("baseline_daily_quantity")) * 100
        ).otherwise(F.lit(None))
    )
    .withColumn(
        "net_lift_pct",
        F.col("incremental_lift_pct") - F.col("cannibalization_pct")
    )
    .withColumn(
        "roi_category",
        F.when(F.col("net_lift_pct") > 50, "High ROI")
         .when(F.col("net_lift_pct") > 20, "Medium ROI")
         .when(F.col("net_lift_pct") > 0, "Low ROI")
         .otherwise("Negative ROI")
    )
    .withColumn("computed_at", F.current_timestamp())
)

print(f"\nPromotion lift analysis complete: {df_promotion_lift.count():,} promo-product combinations")
print("\nSample results:")
df_promotion_lift.orderBy(F.desc("incremental_lift_pct")).show(10, truncate=False)

In [ ]:
# Save promotion lift to Gold layer
print("\nSaving gold_promotion_lift...")
save_gold(df_promotion_lift, "gold_promotion_lift")

---
## Summary & Insights

In [ ]:
print("\n" + "="*60)
print("PROMOTION EFFECTIVENESS ANALYSIS COMPLETE")
print("="*60)

# Elasticity summary
print("\n--- Price Elasticity Summary ---")
df_price_elasticity.groupBy("elasticity_category").count().orderBy(F.desc("count")).show()

print("\nMost price-sensitive products (highest elasticity):")
df_price_elasticity.orderBy(F.desc("elasticity_coefficient")).select(
    "product_name", "product_category", "elasticity_coefficient", "elasticity_category"
).show(5, truncate=False)

print("\nLeast price-sensitive products (lowest elasticity):")
df_price_elasticity.orderBy(F.asc("elasticity_coefficient")).select(
    "product_name", "product_category", "elasticity_coefficient", "elasticity_category"
).show(5, truncate=False)

# Promotion lift summary
print("\n--- Promotion Lift Summary ---")
df_promotion_lift.groupBy("roi_category").count().orderBy(F.desc("count")).show()

print("\nTop performing promotions (highest net lift):")
df_promotion_lift.orderBy(F.desc("net_lift_pct")).select(
    "promo_code", "product_name", "discount_type", 
    "incremental_lift_pct", "cannibalization_pct", "net_lift_pct", "roi_category"
).show(5, truncate=False)

print("\nPromotions with high cannibalization:")
df_promotion_lift.filter(F.col("cannibalization_pct") > 20).orderBy(F.desc("cannibalization_pct")).select(
    "promo_code", "product_name", "discount_type",
    "incremental_lift_pct", "cannibalization_pct", "net_lift_pct"
).show(5, truncate=False)

print("\nGold tables created:")
print(f"  - {GOLD_DB}.gold_price_elasticity")
print(f"  - {GOLD_DB}.gold_promotion_lift")
print("\nUse these tables for Power BI dashboards and further analysis.")